# Fine Tuning Experiment: OpenAI's GPT-3.5 vs Llama 2 on Replicate

## Introduction

This notebook shows how to compare different fine tuned versions of the same model on OpenAI and Replicate. For this example, we'll compare GPT-3.5 to Llama 2 70b, fine tuned to generate SQL queries from natural language questions.

## Installations

You may need to install `prompttools`, `replicate`, and HuggingFace's `datasets` to complete this example.

In [ ]:
# !pip install --quiet --force-reinstall prompttools datasets replicate

## Setup imports and API keys

First, we'll need to set our API keys. If we are in DEBUG mode, we don't need to use a real OpenAI key, so for now we'll set them to empty strings.

In [1]:
import os

os.environ["DEBUG"] = ""  # Set this to "" to call OpenAI's API
os.environ["OPENAI_API_KEY"] = ""  # Insert your key here

Then we'll import the relevant `prompttools` modules to setup our experiment.

In [2]:
from typing import Dict, List
from datasets import load_dataset
from prompttools.experiment import OpenAIChatExperiment

## Preprocess Data

For this experiment, we'll use the `wikisql` dataset from HuggingFace. We can remove some of the metadata from the inputs to create fine tuning data for gpt-3.5 and Llama 2.

In [3]:
dataset = load_dataset("wikisql")

Found cached dataset wikisql (/Users/stevenkrawczyk/.cache/huggingface/datasets/wikisql/default/0.1.0/7037bfe6a42b1ca2b6ac3ccacba5253b1825d31379e9cc626fc79a620977252d)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from tqdm import tqdm

filtered_set = []

for entry in tqdm(dataset['train']):
    if entry['phase'] == 1:
        modified_entry = {}
        modified_entry['question'] = entry['question']
        modified_entry['sql'] = entry['sql']['human_readable']
        modified_entry['table'] = {}
        modified_entry['table']['header'] = entry['table']['header']
        modified_entry['table']['rows'] = entry['table']['rows']
        modified_entry['table']['types'] = entry['table']['types']
        filtered_set.append(modified_entry)

About 500 rows is the recommended minimum number for fine-tuning, so we'll same 500 rows from our dataset.

In [ ]:
import random

tuning_set = random.sample(filtered_set, 500)

# Fine tuning GPT-3.5 on OpenAI

First, we'll create and upload a file for our fine tuning rows, and fine tune the model. The file processing may take a few minutes, and the fine tuning may take a few hours.

In [ ]:
import json

gpt_fine_tuning_rows = []
for entry in tuning_set:
    current_row = {'messages': [
        {"role": "system", "content": "You are a text-to-SQL helper. " + \
                                      "Given the following table, produce " + \
                                      "valid SQL to answer the user question:" + \
                                      str(entry['table'])},
        {"role": "user", "content": entry['question']},
        {"role": "assistant", "content": entry['sql']},
    ]}
    gpt_fine_tuning_rows.append(current_row)

In [ ]:
import json
for row in gpt_fine_tuning_rows:
    with open('gpt_fine_tuning_rows.jsonl', 'a') as f:
        f.write(json.dumps(row) + '\n')

In [ ]:
import openai
import os

openai.File.create(
  file=open("gpt_fine_tuning_rows.jsonl", "rb"),
  purpose='fine-tune'
)


In [ ]:
openai.FineTuningJob.create(training_file="YOUR_FILE_NAME_HERE", model="gpt-3.5-turbo")

In [ ]:
import openai
openai.FineTuningJob.list(limit=10)

## Compare fine tuned model to base model

Once the fine tuning job finishes, we can compare the fine tuned model to the base model in the same experiment, as seen below.

In [4]:
from tqdm import tqdm

filtered_test_set = []

for entry in tqdm(dataset['test']):
    if entry['phase'] == 1:
        filtered_test_set.append(entry)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15878/15878 [00:03<00:00, 4170.29it/s]


In [5]:
models = ["gpt-3.5-turbo", "ft:gpt-3.5-turbo-0613:YOUR_ORG_HERE::YOUR_ID"] 
messages = [
    [
        {"role": "system", "content": "You are a text-to-SQL helper. " + \
                                      "Given the following table, produce " + \
                                      "valid SQL to answer the user question:" + \
                                      str(filtered_test_set[0]['table'])},
        {"role": "user", "content": filtered_test_set[0]['question']},
    ]
]
temperatures = [0.0]
# You can add more parameters that you'd like to test here.

experiment = OpenAIChatExperiment(models, messages, temperature=temperatures)

In [6]:
experiment.run()

In [7]:
experiment.visualize()

,model,messages,response,latency
0,gpt-3.5-turbo,"[{'role': 'system', 'content': 'You are a text-to-SQL helper. Given the following table, produce valid SQL to answer the user question:{'header': ['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team'], 'page_title': 'Toronto Raptors all-time roster', 'page_id': '', 'types': ['text', 'text', 'text', 'text', 'text', 'text'], 'id': '1-10015132-16', 'section_title': 'R', 'caption': 'R', 'rows': [['Aleksandar Radojević', '25', 'Serbia', 'Center', '1999-2000', 'Barton CC (KS)'], ['Shawn Respert', '31', 'United States', 'Guard', '1997-98', 'Michigan State'], ['Quentin Richardson', 'N/A', 'United States', 'Forward', '2013-present', 'DePaul'], ['Alvin Robertson', '7, 21', 'United States', 'Guard', '1995-96', 'Arkansas'], ['Carlos Rogers', '33, 34', 'United States', 'Forward-Center', '1995-98', 'Tennessee State'], ['Roy Rogers', '9', 'United States', 'Forward', '1998', 'Alabama'], ['Jalen Rose', '5', 'United States', 'Guard-Forward', '2003-06', 'Michigan'], ['Terrence Ross', '31', 'United States', 'Guard', '2012-present', 'Washington']], 'name': 'table_10015132_16'}'}, {'role': 'user', 'content': 'What is terrence ross' nationality'}]",SELECT Nationality\nFROM table_10015132_16\nWHERE Player = 'Terrence Ross',1.120881
1,ft:gpt-3.5-turbo-0613:hegel-ai::7ximyJEn,"[{'role': 'system', 'content': 'You are a text-to-SQL helper. Given the following table, produce valid SQL to answer the user question:{'header': ['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team'], 'page_title': 'Toronto Raptors all-time roster', 'page_id': '', 'types': ['text', 'text', 'text', 'text', 'text', 'text'], 'id': '1-10015132-16', 'section_title': 'R', 'caption': 'R', 'rows': [['Aleksandar Radojević', '25', 'Serbia', 'Center', '1999-2000', 'Barton CC (KS)'], ['Shawn Respert', '31', 'United States', 'Guard', '1997-98', 'Michigan State'], ['Quentin Richardson', 'N/A', 'United States', 'Forward', '2013-present', 'DePaul'], ['Alvin Robertson', '7, 21', 'United States', 'Guard', '1995-96', 'Arkansas'], ['Carlos Rogers', '33, 34', 'United States', 'Forward-Center', '1995-98', 'Tennessee State'], ['Roy Rogers', '9', 'United States', 'Forward', '1998', 'Alabama'], ['Jalen Rose', '5', 'United States', 'Guard-Forward', '2003-06', 'Michigan'], ['Terrence Ross', '31', 'United States', 'Guard', '2012-present', 'Washington']], 'name': 'table_10015132_16'}'}, {'role': 'user', 'content': 'What is terrence ross' nationality'}]",SELECT Nationality FROM table_10015132_16 WHERE Player = Terrence Ross,1.149571


In [8]:
from prompttools.utils import similarity
experiment.evaluate("correctness", similarity.semantic_similarity, expected=[filtered_test_set[0]['sql']['human_readable']] * 2)
experiment.visualize()

,model,messages,response,latency,correctness
0,gpt-3.5-turbo,"[{'role': 'system', 'content': 'You are a text-to-SQL helper. Given the following table, produce valid SQL to answer the user question:{'header': ['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team'], 'page_title': 'Toronto Raptors all-time roster', 'page_id': '', 'types': ['text', 'text', 'text', 'text', 'text', 'text'], 'id': '1-10015132-16', 'section_title': 'R', 'caption': 'R', 'rows': [['Aleksandar Radojević', '25', 'Serbia', 'Center', '1999-2000', 'Barton CC (KS)'], ['Shawn Respert', '31', 'United States', 'Guard', '1997-98', 'Michigan State'], ['Quentin Richardson', 'N/A', 'United States', 'Forward', '2013-present', 'DePaul'], ['Alvin Robertson', '7, 21', 'United States', 'Guard', '1995-96', 'Arkansas'], ['Carlos Rogers', '33, 34', 'United States', 'Forward-Center', '1995-98', 'Tennessee State'], ['Roy Rogers', '9', 'United States', 'Forward', '1998', 'Alabama'], ['Jalen Rose', '5', 'United States', 'Guard-Forward', '2003-06', 'Michigan'], ['Terrence Ross', '31', 'United States', 'Guard', '2012-present', 'Washington']], 'name': 'table_10015132_16'}'}, {'role': 'user', 'content': 'What is terrence ross' nationality'}]",SELECT Nationality\nFROM table_10015132_16\nWHERE Player = 'Terrence Ross',1.120881,0.953111
1,ft:gpt-3.5-turbo-0613:hegel-ai::7ximyJEn,"[{'role': 'system', 'content': 'You are a text-to-SQL helper. Given the following table, produce valid SQL to answer the user question:{'header': ['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team'], 'page_title': 'Toronto Raptors all-time roster', 'page_id': '', 'types': ['text', 'text', 'text', 'text', 'text', 'text'], 'id': '1-10015132-16', 'section_title': 'R', 'caption': 'R', 'rows': [['Aleksandar Radojević', '25', 'Serbia', 'Center', '1999-2000', 'Barton CC (KS)'], ['Shawn Respert', '31', 'United States', 'Guard', '1997-98', 'Michigan State'], ['Quentin Richardson', 'N/A', 'United States', 'Forward', '2013-present', 'DePaul'], ['Alvin Robertson', '7, 21', 'United States', 'Guard', '1995-96', 'Arkansas'], ['Carlos Rogers', '33, 34', 'United States', 'Forward-Center', '1995-98', 'Tennessee State'], ['Roy Rogers', '9', 'United States', 'Forward', '1998', 'Alabama'], ['Jalen Rose', '5', 'United States', 'Guard-Forward', '2003-06', 'Michigan'], ['Terrence Ross', '31', 'United States', 'Guard', '2012-present', 'Washington']], 'name': 'table_10015132_16'}'}, {'role': 'user', 'content': 'What is terrence ross' nationality'}]",SELECT Nationality FROM table_10015132_16 WHERE Player = Terrence Ross,1.149571,0.952774


For our example, both the base model and fine tuned model do a good job at returning only SQL, but the tuned model is marginally closer to the correct answer and has better syntax.

# Replicate: Llama 2 70 B

Since Llama 2 has a smaller context window, will restict the size of the rows we use for fine-tuning from the original set.

In [ ]:
filtered_tuning_set = [entry for entry in tuning_set if len(str(entry['table'])) < 2000]

In [ ]:
import json

replicate_fine_tuning_rows = []
for entry in filtered_tuning_set:
    current_row = {'prompt': entry['question'] + \
                             "\nGiven the following table, produce " + \
                             "valid SQL to answer the question:" + \
                              str(entry['table']) ,
                   'completion': entry['sql']}
    replicate_fine_tuning_rows.append(current_row)


In [ ]:
import json
for row in replicate_fine_tuning_rows:
    with open('replicate_fine_tuning_rows.jsonl', 'a') as f:
        f.write(json.dumps(row) + '\n')

## Run fine tuning

For replicate, you will need to upload your fine-tuning rows to an accessible URL. For our example, we uploaded the file to an S3 bucket and used a signed URL to provide access.

The fine tuning may take a few hours.

In [23]:
import replicate

os.environ["REPLICATE_API_TOKEN"] = ""

base_model = "meta/llama-2-70b-chat:35042c9a33ac8fd5e29e27fb3197f33aa483f72c2ce3b0b9d201155c7fd2a287"

In [ ]:
training = replicate.trainings.create(
  version=base_model,
  input={
    "train_data": "S3-URL-HERE",
  },
  destination="YOUR-REPLICATE-ID/prompttools-example"
)

Wait for the fine tuning job to finish, then test that the model works.

In [ ]:
training.reload()
if training.status == "succeeded":
    print(training.output)

In [ ]:
output = replicate.run(
  training.output["version"],
  input={"prompt": filtered_tuning_set[0]['question'] + \
                   "\nGiven the following table, produce " + \
                   "valid SQL to answer the question:" + \
                    str(filtered_tuning_set[0]['table'])}
)
for s in output:
    print(s, end="", flush=True)

## Compared tuned Llama model to base model

Now, we can set up a replicate experiment between the fine tuned and base Llama models.

In [20]:
from prompttools.experiment import ReplicateExperiment
llama = "YOUR-REPLICATE-ID/prompttools-example:VERSION"
models = [llama, base_model]
input_kwargs = {"prompt": [filtered_test_set[0]['question'] + \
                           "\nGiven the following table, produce " + \
                           "valid SQL to answer the question:" + \
                            str(filtered_test_set[0]['table'])]}
model_specific_kwargs = {llama: {}, base_model: {}}

replicate_experiment = ReplicateExperiment(models, input_kwargs, model_specific_kwargs)

In [21]:
replicate_experiment.run()
replicate_experiment.visualize()

,model_version,prompt,response,latency
0,hegelai/llama-test-large:bb9b696ca41ac10be77c786fdad0f7c17ed80a67dc14879ca66ec02f4d5e8caf,"What is terrence ross' nationality\nGiven the following table, produce valid SQL to answer the question:{'header': ['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team'], 'page_title': 'Toronto Raptors all-time roster', 'page_id': '', 'types': ['text', 'text', 'text', 'text', 'text', 'text'], 'id': '1-10015132-16', 'section_title': 'R', 'caption': 'R', 'rows': [['Aleksandar Radojević', '25', 'Serbia', 'Center', '1999-2000', 'Barton CC (KS)'], ['Shawn Respert', '31', 'United States', 'Guard', '1997-98', 'Michigan State'], ['Quentin Richardson', 'N/A', 'United States', 'Forward', '2013-present', 'DePaul'], ['Alvin Robertson', '7, 21', 'United States', 'Guard', '1995-96', 'Arkansas'], ['Carlos Rogers', '33, 34', 'United States', 'Forward-Center', '1995-98', 'Tennessee State'], ['Roy Rogers', '9', 'United States', 'Forward', '1998', 'Alabama'], ['Jalen Rose', '5', 'United States', 'Guard-Forward', '2003-06', 'Michigan'], ['Terrence Ross', '31', 'United States', 'Guard', '2012-present', 'Washington']], 'name': 'table_10015132_16'}",SELECT Nationality FROM table_ SELECT Nationality FROM table_1001513 SELECT Nationality FROM table_10015132_16 WHERE Player = Nationality FROM table_10015132_16 WHERE Player = 'Terrence Ross'; Nationality FROM table_10015132_16 WHERE Player = 'Terrence Ross';,0.291232
1,meta/llama-2-70b-chat:35042c9a33ac8fd5e29e27fb3197f33aa483f72c2ce3b0b9d201155c7fd2a287,"What is terrence ross' nationality\nGiven the following table, produce valid SQL to answer the question:{'header': ['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team'], 'page_title': 'Toronto Raptors all-time roster', 'page_id': '', 'types': ['text', 'text', 'text', 'text', 'text', 'text'], 'id': '1-10015132-16', 'section_title': 'R', 'caption': 'R', 'rows': [['Aleksandar Radojević', '25', 'Serbia', 'Center', '1999-2000', 'Barton CC (KS)'], ['Shawn Respert', '31', 'United States', 'Guard', '1997-98', 'Michigan State'], ['Quentin Richardson', 'N/A', 'United States', 'Forward', '2013-present', 'DePaul'], ['Alvin Robertson', '7, 21', 'United States', 'Guard', '1995-96', 'Arkansas'], ['Carlos Rogers', '33, 34', 'United States', 'Forward-Center', '1995-98', 'Tennessee State'], ['Roy Rogers', '9', 'United States', 'Forward', '1998', 'Alabama'], ['Jalen Rose', '5', 'United States', 'Guard-Forward', '2003-06', 'Michigan'], ['Terrence Ross', '31', 'United States', 'Guard', '2012-present', 'Washington']], 'name': 'table_10015132_16'}",The nationality of Terrence Ross is United States.,0.302811


In [22]:
replicate_experiment.evaluate("correctness", similarity.semantic_similarity, expected=[filtered_test_set[0]['sql']['human_readable']] * 2)
replicate_experiment.visualize()

,model_version,prompt,response,latency,correctness
0,hegelai/llama-test-large:bb9b696ca41ac10be77c786fdad0f7c17ed80a67dc14879ca66ec02f4d5e8caf,"What is terrence ross' nationality\nGiven the following table, produce valid SQL to answer the question:{'header': ['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team'], 'page_title': 'Toronto Raptors all-time roster', 'page_id': '', 'types': ['text', 'text', 'text', 'text', 'text', 'text'], 'id': '1-10015132-16', 'section_title': 'R', 'caption': 'R', 'rows': [['Aleksandar Radojević', '25', 'Serbia', 'Center', '1999-2000', 'Barton CC (KS)'], ['Shawn Respert', '31', 'United States', 'Guard', '1997-98', 'Michigan State'], ['Quentin Richardson', 'N/A', 'United States', 'Forward', '2013-present', 'DePaul'], ['Alvin Robertson', '7, 21', 'United States', 'Guard', '1995-96', 'Arkansas'], ['Carlos Rogers', '33, 34', 'United States', 'Forward-Center', '1995-98', 'Tennessee State'], ['Roy Rogers', '9', 'United States', 'Forward', '1998', 'Alabama'], ['Jalen Rose', '5', 'United States', 'Guard-Forward', '2003-06', 'Michigan'], ['Terrence Ross', '31', 'United States', 'Guard', '2012-present', 'Washington']], 'name': 'table_10015132_16'}",SELECT Nationality FROM table_ SELECT Nationality FROM table_1001513 SELECT Nationality FROM table_10015132_16 WHERE Player = Nationality FROM table_10015132_16 WHERE Player = 'Terrence Ross'; Nationality FROM table_10015132_16 WHERE Player = 'Terrence Ross';,0.291232,0.818023
1,meta/llama-2-70b-chat:35042c9a33ac8fd5e29e27fb3197f33aa483f72c2ce3b0b9d201155c7fd2a287,"What is terrence ross' nationality\nGiven the following table, produce valid SQL to answer the question:{'header': ['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team'], 'page_title': 'Toronto Raptors all-time roster', 'page_id': '', 'types': ['text', 'text', 'text', 'text', 'text', 'text'], 'id': '1-10015132-16', 'section_title': 'R', 'caption': 'R', 'rows': [['Aleksandar Radojević', '25', 'Serbia', 'Center', '1999-2000', 'Barton CC (KS)'], ['Shawn Respert', '31', 'United States', 'Guard', '1997-98', 'Michigan State'], ['Quentin Richardson', 'N/A', 'United States', 'Forward', '2013-present', 'DePaul'], ['Alvin Robertson', '7, 21', 'United States', 'Guard', '1995-96', 'Arkansas'], ['Carlos Rogers', '33, 34', 'United States', 'Forward-Center', '1995-98', 'Tennessee State'], ['Roy Rogers', '9', 'United States', 'Forward', '1998', 'Alabama'], ['Jalen Rose', '5', 'United States', 'Guard-Forward', '2003-06', 'Michigan'], ['Terrence Ross', '31', 'United States', 'Guard', '2012-present', 'Washington']], 'name': 'table_10015132_16'}",The nationality of Terrence Ross is United States.,0.302811,0.562899


Our fine tuned model did significantly better than the base model at producing SQL in response to the prompt, but neither is as good as gpt-3.5 base or fine tuned.

# Conclusion

GPT-3.5 does better at producing SQL, especially when fine tuned. One advantage is the larger context window, which allows us to tune on larger SQL tables and provide more context at query time. Another advantage is the power of the underlying model, which is already quite good at the task.